# 다양한 모델링 시도

## Hyperparameter

In [ ]:
CFG = {
    'WIDTH':48,
    'HEIGHT':72,
    'EPOCHS':25,
    'LEARNING_RATE':1e-3,
    'BATCH_SIZE':128,
    'SEED':41
}

## AutoEncoder 변형
* BottleNeck 구조의 encoder, decoder 사이에 autoencoder를 추가

In [ ]:
class Autoencoder_plus(nn.Module):
    def __init__(self):
        super(Autoencoder_plus, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(CFG['HEIGHT']*CFG['WIDTH'], 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(), 
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU()
        )
        
        self.middle = nn.Sequential(                                            # encoder, decoder 사이 autoencoder 추가
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU())

        self.decoder = nn.Sequential(
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Linear(1024, CFG['HEIGHT']*CFG['WIDTH']),
        )
        
    def forward(self, x):
        x = x.view(-1, CFG['HEIGHT']*CFG['WIDTH'])
        x = self.encoder(x)
        x = self.middle(x)
        x = self.decoder(x)
        x = x.view(-1, 1, CFG['HEIGHT'], CFG['WIDTH'])
        return x

## CNN Model1 → Conv1D + Decoder(linear) 
* 단면 단위의 정보가 중요하므로 이미지를 1차원으로 펴서 정보를 추출

In [ ]:
class CNN_Model1(nn.Module):
  def __init__(self):
    super(CNN_Model1, self).__init__()
    # Encoder
    self.cnn_layer1 = nn.Sequential(nn.Conv1d(1, 128, kernel_size=3, stride=1),
                                 nn.BatchNorm1d(128),
                                 nn.ReLU(),
                                 nn.MaxPool1d(2))
    self.cnn_layer2 = nn.Sequential(nn.Conv1d(128, 64, kernel_size=3, stride=1),
                                 nn.BatchNorm1d(64),
                                 nn.ReLU(),
                                 nn.MaxPool1d(2))
    self.cnn_layer3 = nn.Sequential(nn.Conv1d(64, 1, kernel_size=3, stride=1),
                                 nn.BatchNorm1d(1),
                                 nn.ReLU())
    
    self.flatten = nn.Flatten()

    # Decoder                                                      
    self.decoder = nn.Sequential(
            nn.Linear(860, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Linear(1024, CFG['HEIGHT']*CFG['WIDTH']),
        )
    
  def forward(self, x):
    x = x.view(-1, 1, CFG['HEIGHT']*CFG['WIDTH'])
    x = self.cnn_layer1(x)
    x = self.cnn_layer2(x)
    x = self.cnn_layer3(x)
    x = self.flatten(x)
    x = self.decoder(x)                             
    x = x.view(-1, 1, CFG['HEIGHT'], CFG['WIDTH'])

    return x 

## CNN Model2 → Conv1D + ConvTranspose1D

In [ ]:
class CNN_Model2(nn.Module):
  def __init__(self):
    super(CNN_Model2, self).__init__()
    # Encoder
    self.cnn_layer1 = nn.Sequential(nn.Conv1d(in_channels=1, out_channels=256, kernel_size=5, stride=1, padding=2),
                                    nn.BatchNorm1d(256),
                                    nn.ReLU(),
                                    nn.MaxPool1d(2,2))
    self.cnn_layer2 = nn.Sequential(nn.Conv1d(in_channels=256, out_channels=128, kernel_size=5, stride=1, padding=2),
                                    nn.BatchNorm1d(128),
                                    nn.ReLU(),
                                    nn.MaxPool1d(2,2))
    self.cnn_layer3 = nn.Sequential(nn.Conv1d(in_channels=128, out_channels=64, kernel_size=5, stride=1, padding=2),
                                    nn.BatchNorm1d(64),
                                    nn.ReLU(),
                                    nn.MaxPool1d(2,2))
    
    # Decoder
    self.cnn_layer4 = nn.Sequential(nn.ConvTranspose1d(in_channels=64, out_channels=128, kernel_size=5, stride=2, padding=2),
                                    nn.BatchNorm1d(128),
                                    nn.ReLU())
    self.cnn_layer5 = nn.Sequential(nn.ConvTranspose1d(in_channels=128, out_channels=256, kernel_size=5, stride=2, padding=2),
                                    nn.BatchNorm1d(256),
                                    nn.ReLU())
    self.cnn_layer6 = nn.Sequential(nn.ConvTranspose1d(in_channels=256, out_channels=1, kernel_size=5, stride=2, padding=2),
                                    nn.BatchNorm1d(1),
                                    nn.ReLU(),
                                    nn.Dropout(0.2))
    self.dense = nn.Sequential(nn.Linear(3449, 3456),
                               nn.ReLU(3456)
                               )
    
  def forward(self, x):                                              
    x = x.view(-1,1, CFG['HEIGHT']*CFG['WIDTH'])                                # 배치 크기, 채널, 높이 * 너비 = (128, 1, 72*48)                 
    x = self.cnn_layer1(x)
    x = self.cnn_layer2(x)
    x = self.cnn_layer3(x)
    x = self.cnn_layer4(x)
    x = self.cnn_layer5(x)
    x = self.cnn_layer6(x)
    x = self.dense(x)
    x = x.view(-1,1, CFG['HEIGHT'],CFG['WIDTH'])
    return x

## CNN Model3 → Conv2D + ConvTranspose 2D
* convolution layer로 이미지의 공간적인 정보를 추출

In [ ]:
class CNN_Model3(nn.Module):
  def __init__(self):
    super(CNN_Model3, self).__init__()
    self.encoder = nn.Sequential(nn.Conv2d(1, 256, kernel_size=4, stride=2, padding=1),
                                 nn.BatchNorm2d(256),
                                 nn.ReLU(),
                                 nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
                                 nn.BatchNorm2d(512),
                                 nn.ReLU(),
                                 nn.Conv2d(512, 1024, kernel_size=4, stride=2, padding=1),
                                 nn.BatchNorm2d(1024),
                                 nn.ReLU()                                           
                                 )
    
    self.dropout = nn.Dropout(0.2)
    
    self.decoder = nn.Sequential(
                                nn.ConvTranspose2d(1024, 512, kernel_size=4, stride=2, padding=1),
                                nn.BatchNorm2d(512), 
                                nn.ReLU(),
                                nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
                                nn.BatchNorm2d(256), 
                                nn.ReLU(),
                                nn.ConvTranspose2d(256, 1, kernel_size=4, stride=2, padding=1)                                
                                )
    
  def forward(self, x):
    #print('original', x.shape)        
    x = self.encoder(x)
    x = self.dropout(x)
    #print('enc', x.shape)             
    x = self.decoder(x)
    #print("dec", x.shape)
    return x